## 1、获取指定日期的活跃人数

![](img1/1.png)

表结构：

1、id  主键、自增

2、 channel 用户来自于哪个频道 

3、userid 用户id 不一定是int (可能来自于合作频道) (索引)

4、username 登录名 

5、logintime  登录时间 (索引)

![](img1/2.png)

- 需求：获取指定日期的活跃人数

  获取2019年4月1日的活跃人数

  写法1：

  ```sql
  select count(*),DATE_FORMAT(logintime,'%Y-%m-%d')
  from user_login
  where DATE_FORMAT(logintime,'%Y-%m-%d')='2019-04-01';
  ```

  写法2：

  ```sql
  select count(*),DATE_FORMAT(logintime,'%Y-%m-%d')
  from user_login
  where DATEDIFF(logintime,'2019-04-01')=0
  ```

  Explain:描述sql查询的效率，以下描述效率从高到低

  ```sql
  system > const > eq_ref > ref > fulltext > ref_or_null > index_merge > unique_subquery > index_subquery > range > index > ALL
  ```

  写法3：最优写法

  ```sql
  explain 
  select count(*),DATE_FORMAT(logintime,'%Y-%m-%d')
  from user_login
  where logintime>='2019-04-01' and logintime<'2019-04-02';
  ```

## 2、统计指定日期注册新用户的次日登录人数

表名：user_login

1、id  主键、自增

2、 channel 用户来自于哪个频道 

3、userid 用户id 不一定是int (可能来自于合作频道) (索引)

4、username 登录名 

5、logintime  登录时间 (索引)

![](img1/3.png)

表名：user_info

1、userid  主键、用户ID

2、 addtime  注册时间  （索引）

![](img1/4.png)

- 指定日期注册用户的次日登录人数

  举例：2019-3-05

  1、 2019-03-05 注册的用户

  2、次日（也就是2019-03-06）登录的用户
  - 指定日期的新增用户总数

     select count(*) from user_infoaddtimeaddtime'2019-03-06

  - 获取指定日期 +1 的登录数以及userid(这要注意的是：一个用户可能不止登陆一次)

    ```sql
    select userid,count(*) from user_login  where 
    logintime>='2019-03-06' and logintime<'2019-03-07'
    GROUP BY userid
    ```

    因此：

    ```sql
    select count(DISTINCT userid) from user_login  where 
    logintime>='2019-03-06' and logintime<'2019-03-07'
    ```

  - 过滤刚才的结果:

    刚才是获取全部的，实际上我们要获取指定用户登录数

    ```sql
    select count(DISTINCT userid) as c1 from user_login  where 
    logintime>='2019-03-06' and logintime<'2019-03-07' and userid in 
    (
      select userid from user_info where addtime>='2019-03-05' and addtime<'2019-03-06'
    )
    ```

## 3、统计指定日期范围内用户的登录总次数、每天登录次数

表名：user_login

1、id  主键、自增

2、 channel 用户来自于哪个频道 

3、userid 用户id 不一定是int (可能来自于合作频道) (索引)

4、username 登录名 

5、logintime  登录时间 (索引)

![](img1/5.png)

表名：user_info

1、userid  主键、用户ID

2、 addtime  注册时间  （索引）

![](img1/6.png)

- 需求

  统计指定日期范围内用户的登录总次数

  譬如 2019-03-05~~~~~2019-03-10  .

  注意：一个用户一天内不止登录一次

- 获取总次数

  ```sql
  select userid,count(*)  from user_login where logintime>='2019-03-05' and logintime<'2019-03-11'
  group by userid
  ```

- 获取每天

  ```sql
  select userid,DATE_FORMAT(logintime,'%Y%m%d') as dd,count(*) as cc from user_login where logintime>='2019-03-05' and logintime<'2019-03-11'
  and userid='u1001997'
  group by userid,DATE_FORMAT(logintime,'%Y%m%d') 
  ```

## 4、统计指定日期范围内每天都登录的用户(连续x天都登录、加行号)

- 统计指定日期范围内用户的登录总次数、每天登录的次数

  ```sql
  select userid,DATE_FORMAT(logintime,'%Y%m%d'),count(*) from user_login 
  where logintime>='2019-03-05' and logintime<'2019-03-11'
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d')
  ```

- **指标分析**

  5号-10号

  1、我们需要的不是次数

  2、而是每天都有登录的用户

- 修改

  ```sql
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') from user_login 
  where logintime>='2019-03-05' and logintime<'2019-03-11'
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d')
  ```

- 定义一些初始的变量

  我们不需要用存储过程，直接 用select定义即可

  select @pre:='',@rownum:=0

- 最终SQL

  ```sql
  select userid,d,IF(@pre=userid,@rownum:=@rownum+1,@rownum:=1) as rownum ,@pre:=userid   from (
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d
   from user_login 
  where logintime>='2019-03-05' and logintime<'2019-03-11'
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d'))  a,(select @pre:='',@rownum:=0 ) b
  ```

## 5、活跃用户的定义、模拟日活用户数、7天日活数统计

- **活跃用户**

  不同的行业、产品都有不同的定义

   1、门户网站： 一周内至少登录并评论或点赞 一次

   2、游戏类。每天登录 1-2次，游戏在线时长超过xxx分钟

   3、社交应用 ：每天都有登录，且超过1-2次

- 定义

  凡是 每天登录超过2次的（>=2）

   计入我们的日活用户

- 先获取某一天的日活数

  以2019-03-02 为例，我们要获取 该日的登录情况

  ```sql
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d,count(*) as logintcount
   from user_login 
  where logintime>='2019-03-02' and logintime<'2019-03-03'
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d')
  ```

- **过滤**

  过滤出 登录次数大于等于2的人数

  ```sql
  select count(*) from (
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d,count(*) as logintcount
   from user_login 
  where logintime>='2019-03-02' and logintime<'2019-03-03'
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d') ) a where a.logintcount>=2
  ```

- 实际上我们 后台统计往往 得到一天的日活数是不够的，譬如 我们要得到 7天的日活数

  ```sql
  select  d,count(*) from (
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d,count(*) as logintcount
   from user_login 
  where logintime>='2019-03-02' and logintime<DATE_ADD('2019-03-02',INTERVAL 7 day)
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d') ) a where a.logintcount>=2
  GROUP BY d
  ```

## 6、统计指定日期日活用户的日增长率（一条SQL）

- **活跃用户**

  不同的行业、产品都有不同的定义

   1、门户网站： 一周内至少登录并评论或点赞 一次

   2、游戏类。每天登录 1-2次，游戏在线时长超过xxx分钟

   3、社交应用 ：每天都有登录，且超过1-2次

  第5节凡是每天登录超过2次的（>=2），计入我们的日活用户

- 最终结果

  从2019-03-02往后推7天的 日活人数

  ```sql
  select  d,count(*) from (
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d,count(*) as logintcount
   from user_login 
  where logintime>='2019-03-02' and logintime<DATE_ADD('2019-03-02',INTERVAL 7 day)
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d') ) a where a.logintcount>=2
  GROUP BY d
  ```

- 求3.2-----3.8 的日活  日增长率

  实际统计时，要多算出来2天 (头尾各加一天)

![](img1/7.png)

- 具体的SQL

  ```sql
  select DATE_ADD(a2.d,INTERVAL 1 day),a2.dau2,dau2/dau1 from (
  select a1.*, if(@pre='',0,@pre) as dau2,@pre:=dau1,@rownum:=@rownum+1 as rownum from 
  (select  d,count(*) as dau1 from (
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d,count(*) as logintcount
   from user_login 
  where logintime>='2019-03-01' and logintime<DATE_ADD('2019-03-02',INTERVAL 8 day)
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d') ) a where a.logintcount>=2
  GROUP BY d order by d desc ) a1,(select @pre:='',@rownum:=0) b1 ) a2 where a2.rownum>2 
  ```

## 7、统计月活人数、日活缓存表的使用

- 活跃用户

  不同的行业、产品都有不同的定义

   1、门户网站： 一周内至少登录并评论或点赞 一次

   2、游戏类。每天登录 1-2次，游戏在线时长超过xxx分钟

   3、社交应用 ：每天都有登录，且超过1-2次

凡是每天登录超过2次的（>=2），计入我们的日活用户

- 回顾上次的SQL

  获取指定日期范围(天数)的日活人数。（以下是2月份日活人数）

  ```sql
  select  d,count(*) as dau1 from (
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d,count(*) as logintcount
   from user_login 
  where logintime>='2019-02-01' and logintime<'2019-03-01'  
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d') ) a where a.logintcount>=2
  GROUP BY d order by d desc 
  ```

- 月活

  累加日活的人数

  ```sql
  select DATE_FORMAT(d,'%Y%m'), sum(dau1) from (
   select  d,count(*) as dau1 from (
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d,count(*) as logintcount
   from user_login 
  where logintime>='2019-01-01' and logintime<'2019-04-01'  
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d')  ) a where a.logintcount>=2
  GROUP BY d order by d desc ) a 
  GROUP BY DATE_FORMAT(d,'%Y%m')
  ```

- **缓存表**

  实际上过去的日活人数是不会变了。

  所以一般我们会创建一个 表，存各个天数的日活数据

  ```sql
  insert into user_dau(d,c) 
   select d,count(*) as dau1 from (
  select  userid,DATE_FORMAT(logintime,'%Y%m%d') as d,count(*) as logintcount
   from user_login 
  where logintime>='2019-02-01' and logintime<'2019-03-01'  
  GROUP BY userid,DATE_FORMAT(logintime,'%Y%m%d') ) a where a.logintcount>=2
  GROUP BY d order by d desc
  ```

- 保证能够反复插入

  ON DUPLICATE KEY UPDATE c=values(c)

## 8、留存率、指定日期的次日留存率计算

- **留存率**

  某天进来一批新用户，有的用了一下就再也没回来过，有的用过几次后也离开了，还有些在一定时间段里，能够一直持续活跃使用，这些一直保持活跃的我们称之为留存用户。留存用户占这批次新增用户的比例，就是留存率。

  p一般有: 

  次日留存率：（当天新增的用户中，在注册的第2天还登录的用户数）/第一天新增总用户数；

  第3日留存率：（第一天新增用户中，在注册的第3天还有登录的用户数）/第一天新增总用户数；

  第7日留存率：（第一天新增的用户中，在注册的第7天还有登录的用户数）/第一天新增总用户数；

  第30日留存率：（第一天新增的用户中，在注册的第30天还有登录的用户数）/第一天新增总用户数。

- 举例

  假设我们要获取

  2019-03-01 的次日留存率

  分两步

  1、计算2019-03-01的注册人数

  2、计算2019-03-02的登陆人数

- SQL

  计算2019-03-01的注册人数

  ```sql
  select count(*) from user_info where addtime>='2019-03-01'
  and addtime
  ```

- 计算2019-03-02的登陆人数

  ```sql
  select count(*) from (
  select userid,count(*)  from user_login where logintime>='2019-03-02' and logintime<'2019-03-03'
  group by userid  ) a
  ```

  这样是不够的，因为必须过滤出3-1号注册的人

  ```sql
  select count(*) from (
  select userid,count(*)  from user_login where logintime>='2019-03-02' and logintime<'2019-03-03'
  and userid in (select userid from user_info where addtime>='2019-03-01'
  and addtime<'2019-03-02')
  group by userid  ) a
  ```

- 直接出结果

  ```sql
  select  logincount/regcount   from 
   (select count(*) as regcount from user_info where addtime>='2019-03-01'
  and  addtime<'2019-03-02' ) a ,(select count(*) as logincount from (
  select  userid,count(*)  from user_login where logintime>='2019-03-02' and logintime<'2019-03-03'
  and userid in ( select userid from user_info where addtime>='2019-03-01'
  and  addtime<'2019-03-02')
  GROUP BY userid ) a) b
  ```

## 9、留存率、统计指定日期的连续N天的留存率

- 留存率

  某天进来一批新用户，有的用了一下就再也没回来过，有的用过几次后也离开了，还有些在一定时间段里，能够一直持续活跃使用，这些一直保持活跃的我们称之为留存用户。留存用户占这批次新增用户的比例，就是留存率。

  一般有: (摘录自百度百科)

  次日留存率：（当天新增的用户中，在注册的第2天还登录的用户数）/第一天新增总用户数；

  第3日留存率：（第一天新增用户中，在注册的第3天还有登录的用户数）/第一天新增总用户数；

  第7日留存率：（第一天新增的用户中，在注册的第7天还有登录的用户数）/第一天新增总用户数；

  第30日留存率：（第一天新增的用户中，在注册的第30天还有登录的用户数）/第一天新增总用户数。

- 需求

  以2019-3-1 作为基准

  计算次日、2日、3日。。。。。N日留存率

- 计算2019-03-01的注册人数

  ```sql
  select count(*) from user_info where addtime>='2019-03-01'
  and addtime<'2019-03-02'
  ```

- 计算2019-03-02~~~ 03-08的登陆人数

  ```sql
  select DATE_FORMAT(logintime,'%Y-%m-%d') as ltime,userid,count(*)  
  from user_login where logintime>='2019-03-02' and logintime<'2019-03-09'
  GROUP BY DATE_FORMAT(logintime,'%Y-%m-%d'),userid
  ```

  这样依然是不够的，因为必须过滤出3-1号注册的人

- 完善

  ```sql
  select ltime,count(*) from (
     select DATE_FORMAT(logintime,'%Y-%m-%d') as ltime,userid,count(*)  from user_login where logintime>='2019-03-02' and logintime<'2019-03-09'
  and userid in (select    userid from user_info where addtime>='2019-03-01'
  and  addtime<'2019-03-02')
  GROUP BY DATE_FORMAT(logintime,'%Y-%m-%d'),userid  ) a  
  group by ltime
  ```

- 结果

  ```sql
  select a.ltime, a.logincount/b.regcount from (
    select ltime,count(*) as logincount from (
  select DATE_FORMAT(logintime,'%Y-%m-%d') as ltime,userid,count(*)  from user_login where logintime>='2019-03-02' and logintime<'2019-03-09'
  and userid in (select    userid from user_info where addtime>='2019-03-01'
  and  addtime<'2019-03-02')
  GROUP BY DATE_FORMAT(logintime,'%Y-%m-%d'),userid  ) a  
  group by ltime ) a,(select  count(*) as regcount from user_info where addtime>='2019-03-01'
  and addtime<'2019-03-02') b
  ```

  

KNN

线性回归   a1x1+a2x2+.....a10x10 = 销量

逻辑回归  a1x1+a2x2+.....a10x10 = 是不是羊毛[0，1]  0.85  sigmoid(线性回归的结果) = [0，1]   0.85    

二分类问题

决策树  分类算法  信息熵    

随机森林   GBDT/Xgboost

kmeans   没有目标



分类算法   KNN   逻辑回归  决策树

回归  线性回归



训练集 测试集划分

如何判断算法靠不靠谱   MAE  RMSE   误差角度

分类问题  准确率     精准率 召回率 f1-score  auc





数据预处理/特征工程

缺失值处理 极值处理  归一化 标准化



SQL

union/union all  取并集 union去重  union all不去重

union或union all前后查询出的字段数量必须一致



cross join  inner join   

左外 右外链接   在查询涉及链接问题的时候 不要不写链接条件

**简单case表达式**

```sql
case <表达式>
	when <表达式1> then <表达式4>
	when <表达式2> then <表达式5>
	when <表达式3> then <表达式6>
	。。。
	else <表达式>
end
```

**搜索case表达式**

```sql
case
	when <求值表达式1> then <表达式4>
	when <求值表达式2> then <表达式5>
	when <求值表达式3> then <表达式6>
	。。。
	else <表达式>
end
```



### 行号

会话变量

set @变量名字=变量的值

set @变量名字:=变量的值

select @变量名字

select @ 变量名字:=变量值

IF(表达式，表达式返回true执行的，表达式返回false执行的)

### 取出前N条

- 除了数字类型的字段可以排序，字符串实际上也可以排序，通过order by 对字符型的字段进行排序 也可以实现分组的效果

- **IF函数**

  **IF(条件,表达式1，表达式2)**

  **如果条件成立 则返回表达式1，否则是2**

- 当前案例可以做相关扩展

  - 同一个地区 销量前n个用户
  - 在某个级别的用户中，积分排在前面的

### 复杂sql 执行顺序

From->where->group by->having->select->order by

mysql的 ifnull函数

IFNULL(expr1,expr2) 

expr1 不为 NULL，则 IFNULL() 的返回值为 expr1; 否则其返回值为 expr2

- 自连接

  ```sql
  -- 自连接
  select a.u_name as username,b.u_name as refer from webusers a 
  INNER JOIN webusers b 
  on a.p_id = b.u_id
  
  -- 相同推荐人的用户名字 拼接到一起
  SELECT GROUP_CONCAT(u_name ORDER BY u_id DESC SEPARATOR '|') as u,
  p_id from webusers GROUP BY p_id
  
  select a.users,b.u_name as username FROM 
  (SELECT GROUP_CONCAT(u_name ORDER BY u_id DESC SEPARATOR '|') as users,
  p_id from webusers GROUP BY p_id) a 
  INNER JOIN webusers b   -- 内连接
  WHERE a.p_id = b.u_id   -- 连接条件
  ```

  - GROUP_CONCAT

    通常和group by 一起使用，把相同的分组的字段值连接起来

- 找到重复的数据(添加行号)

  ```sql
  SELECT
  	r_id 
  FROM
  	(
  	SELECT
  		a.*,
  	IF
  		( @tmp = CONCAT( r_content, r_userid ), @rownum := @rownum + 1, @rownum := 1 ) AS rownum,
  		-- 判断CONCAT( r_content, r_userid )和上一条是否一样 如果一样 行号@rownum+1 如果不一样行号从1开始计数
  		@tmp := CONCAT( r_content, r_userid ) -- 给@tmp变量赋值 每一行都是内容和id的拼接
  	FROM
  		(
  		SELECT
  			a.* 
  		FROM
  			reviews a
  			INNER JOIN ( SELECT r_content, r_userid, count(*) FROM reviews GROUP BY r_content, r_userid HAVING count(*)> 1 ) b ON a.r_content = b.r_content 
  			AND a.r_userid = b.r_userid 
  			) a,(
  		SELECT
  			@rownum := 0,   -- 给行号@rownum 用来记录id和评论内容的临时变量@tmp 赋初始值
  			@tmp := '' 
  		) b 
  	) c 
  WHERE
  	rownum >1   -- 筛选出行号>1的 也就是有重复的部分
  ```

  - concat 字符串拼接函数

- 通过unique索引和md5配合避免插入重复的值
  - md5 摘要算法  不管输入的是多长的内容 都会返回32位的16进制串， 如果输入不变，md5之后的结果也不变，如果输入发生一点变化，生成的md5变化也很大，而且md5没有逆向算法
  - 通过mysql提供的md5函数 对 标题和摘要的组合计算md5 作为新闻的唯一标识 news_code 这一列加上Unique索引，当插入的内容标题和摘要相同的是时候 news_code相同 内容无法插入，避免插入重复的值
  - ON DUPLICATE KEY UPDATE 记录输入重复的次数

- 用户消费等级划分

  - ```sql
    -- 通过 在update中加入子查询 和case when 配合 实现多用户级别判断
    update user_level,(select avg(user_total) as avg from user_level) b -- 子查询计算平均值
    set user_rank =
    case
    when round(user_total/avg)>=1 and ROUND(user_total/avg)<2 then '白金用户'
    when round(user_total/avg)>=2 then '黄金用户' 
    ELSE '吃瓜群众'
    END where user_total>=b.avg
    ```

- 挑出部分用户排到前面，不受其它排序条件影响

  ```sql
  select * from user_level order by id in(4,6,2) desc,user_total desc
  ```

- 计算连续签到x天的用户

  ```sql
  SELECT
  	user_name,
  	sign_date 
  FROM
  	(
  	SELECT
  		user_name,
  		sign_date,
  	IF
  		( @pre = user_name AND DATEDIFF( sign_date, @pre_date )= 1, -- 用户名相同，并且签到时间前后差1天的
  		@rownum := @rownum + 1, @rownum := 1 ) AS rnum,   -- 满足上面条件的行号+1 否则从1重新计行号
  		@pre := user_name,                                -- 给pre 和 pre_date变量赋值
  		@pre_date := sign_date 
  	FROM
  		( SELECT user_name, sign_date FROM user_sign GROUP BY user_name, sign_date ORDER BY user_name, sign_date ) a,(
  		SELECT   -- 给会话变量赋初始值
  			@pre := '',
  			@rownum := 0,
  			@pre_date := '' 
  		) b 
  	) a 
  WHERE
  	a.rnum = 3
  ```

  - mysql的函数 datediff(date1,date2)

    返回两个日期之间的天数 注意是 date1-date2

- 子查询去重、获取商品分类最新销售情况

  ```sql
  SELECT
  	a.* 
  FROM
  	prod_sales a
  	INNER JOIN ( SELECT prod_class, prod_id, max( sales_date ) AS sn FROM prod_sales GROUP BY prod_class ) b ON a.prod_class = b.prod_class 
  	AND a.sales_date = b.sn 
  ORDER BY
  	prod_class
  ```

- 多表关联update

  ```sql
  update users_score a INNER JOIN (select max(paymoney) as mp,user_name from users_buy group by user_name ) b on a.user_name=b.user_name 
  set a.user_score= a.user_score+(b.mp*0.1)
  ```

  

- 获取指定日期的活跃人数

  - ```sql
    select count(*),DATE_FORMAT(logintime,'%Y-%m-%d')
    from user_login
    where logintime>='2019-04-01' and logintime<'2019-04-02';
    ```

  - 在处理日期判断的时候 使用 日期字段>,<,= '2019-11-28' 直接比较效率比较高

- 计算次留  前一天的注册用户中选当天的活跃用户

  - ```sql
    select count(DISTINCT userid) as c1 from user_login  where 
    logintime>='2019-03-06' and logintime<'2019-03-07' and userid in 
    (
      select userid from user_info where addtime>='2019-03-05' and addtime<'2019-03-06'
    )
    ```

  - 需要注意，用户的id需要去重，时间圈定的时候 直接用字段和日期比较

- 指定日期范围内用户的登录总次数、每天登录次数

  ```sql
  select userid,DATE_FORMAT(logintime,'%Y-%m-%d') as date,count(*) as count from user_login where logintime>='2019-03-05' and logintime<'2019-03-11' GROUP BY userid,DATE_FORMAT(logintime,'%Y-%m-%d') ORDER BY date,count desc 
  ```

  



- 计算日活增长

  ```sql
  SELECT
  	a1.*,
  IF
  	( @pre = '', 0, @pre ) AS dau2, -- 如果@pre 为空 则赋值为0  如果不为空就是当前的值  起别名dau2
  	@pre := dau1,
  	@rownum := @rownum + 1 AS rownum  -- 行号
  FROM
  	(
  	SELECT
  		d,
  		count(*) AS dau1 
  	FROM
  		(
  		SELECT
  			userid,
  			DATE_FORMAT( logintime, '%Y%m%d' ) AS d,
  			count(*) AS logintcount 
  		FROM
  			user_login 
  		WHERE
  			logintime >= '2019-03-02' 
  		AND logintime < DATE_ADD( '2019-03-02', INTERVAL 7 DAY ) GROUP BY userid, DATE_FORMAT( logintime, '%Y%m%d' )) a WHERE a.logintcount > 2 
  	GROUP BY
  		d 
  	ORDER BY
  		d DESC 
  		) a1,(
  	SELECT
  		@pre := '',    -- 赋初始值
  		@rownum := 0 
  	) b1
  ```



- 7留
  - 第七日留存  通常算法   3月1日注册的 3月7号的时候还剩多少
  - 七日内留存   3月2日 到 3月7日 只要有登录的都算 / 3月1日注册
  - 间隔7日留存  3月8日登录的





### 神策